<a href="https://colab.research.google.com/github/FreNzQuiN/GenerativeAI/blob/main/Stable_Diffusion/MyLatestInlineSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.1 START -- TEXT2IMG SD**

In [ ]:
!pip install diffusers transformers accelerate safetensors compel --upgrade
import torch
from torch import autocast
from diffusers import StableDiffusionXLPipeline, DDIMScheduler #EulerAncestralDiscreteScheduler,
from compel import Compel, ReturnedEmbeddingsType
import matplotlib.pyplot as plt
from PIL import Image
import random, time, math, os
import torch, gc
from tqdm import tqdm


if not os.path.exists("scheduler_config.json"):
  !wget https://huggingface.co/cagliostrolab/animagine-xl-4.0/resolve/main/scheduler/scheduler_config.json

custom_config = True #@param {type:'boolean'}
if os.path.exists("scheduler_config.json") and not custom_config:
  with open("scheduler_config.json", "r") as f:
    scheduler_config = json.load(f)
else:
  scheduler_config = {
    "beta_start": 0.00085,
    "beta_end": 0.012,
    "num_train_timesteps": 1000,
    "steps_offset" : 2,
    "beta_schedule": "scaled_linear",
    "clip_sample": False,
    "set_alpha_to_one": False,
    "_class_name": "DDIMScheduler",
    "_diffusers_version": "0.32.2"
  }

pipe = StableDiffusionXLPipeline.from_pretrained(
    "cagliostrolab/animagine-xl-4.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    custom_pipeline="lpw_stable_diffusion_xl",
    add_watermarker=False,
    )

pipe.scheduler = DDIMScheduler.from_config(scheduler_config)
pipe.to('cuda')


In [ ]:
#@title #**1.2 START -- IMG2IMG SD**
!pip install diffusers transformers accelerate safetensors compel --upgrade
import torch
from torch import autocast
from diffusers import StableDiffusionXLPipeline, DDIMScheduler
from compel import Compel, ReturnedEmbeddingsType
import matplotlib.pyplot as plt
from PIL import Image
import random, time, math, os
import torch, gc
from tqdm import tqdm
import json
import requests
from io import BytesIO

if not os.path.exists("scheduler_config.json"):
    !wget https://huggingface.co/cagliostrolab/animagine-xl-4.0/resolve/main/scheduler/scheduler_config.json

custom_config = True  # @param {type:'boolean'}
if os.path.exists("scheduler_config.json") and not custom_config:
    with open("scheduler_config.json", "r") as f:
        scheduler_config = json.load(f)
else:
    scheduler_config = {
        "beta_start": 0.00085,
        "beta_end": 0.012,
        "num_train_timesteps": 1000,
        "steps_offset": 2,
        "beta_schedule": "scaled_linear",
        "clip_sample": False,
        "set_alpha_to_one": False,
        "_class_name": "DDIMScheduler",
        "_diffusers_version": "0.32.2"
    }

pipe = StableDiffusionXLPipeline.from_pretrained(
    "cagliostrolab/animagine-xl-4.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    custom_pipeline="lpw_stable_diffusion_xl",
    add_watermarker=False,
)

pipe.scheduler = DDIMScheduler.from_config(scheduler_config)
pipe.to('cuda')

In [ ]:
#@title #**2.1 PLAY TEXT2IMG SD**
num=0
prompt_num=0
def disabled_safety_checker(images, clip_input):
    if len(images.shape)==4:
        num_images = images.shape[0]
        return images, [False]*num_images
    else:
        return images, False
pipe.safety_checker = disabled_safety_checker

In [ ]:


w = f"/content/outputs"
if not os.path.exists(w):
    os.makedirs(w)

def generate_image(pipe, prompt, negative_prompt, output_path, width, height, guidance_scale, guidance_rescale, num_inference_steps, seed):
    """Menghasilkan gambar Stable Diffusion."""
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        seed=seed,
        width=width,
        height=height,
        guidance_scale=guidance_scale,
        guidance_rescale=guidance_rescale,
        target_size=(width, height),
        original_size=(4096, 4096),
        num_inference_steps=num_inference_steps,
        truncation=True
    ).images[0]
    image.save(output_path)
    return image

def log_metadata(file_path, start_num, end_num, output_base, width, height, prompt, negative_prompt, guidance_scale, guidance_rescale, num_inference_steps, elapsed_time):
    """Mencatat metadata ke dalam file teks."""
    with open(file_path, "a") as text_file:
        text_file.write(f"\n\nTITLE: {start_num}--{end_num} {output_base} {width}x{height}px\n + {prompt}\n - {negative_prompt}\n guide scale/rescale: {guidance_scale}/{guidance_rescale}\n steps: {num_inference_steps}\nIt took {math.floor(elapsed_time/60)} minutes {math.floor(elapsed_time%60)} seconds!")

amount = 5 #@param {type:'integer'}
#@markdown ---
char = "(1girl, goldenglow \\(arknights\\), arknights \\(series\\))" #@param {type:"string"}
desc_char = "close up, smirk, seducing expression, light blush, her outfit, mature teen, glow eyes, large breast" #@param {type:'string'}
pov = "looking at viewer, from front, depth of field" # @param {"type":"string","placeholder":"dutch angle, looking at viewer, from front, depth of field, close up"}
bg = " colorful, indoors, room, natural scenery" # @param {"type":"string","placeholder":"colorful, indoors, natural scenery"}
style = "masterpiece, high score, best quality, amazing quality, absurdres, [vibrant, very aesthetic, high contrast], masterful composition" # @param {"type":"string","placeholder":"masterpiece, high score, best quality, amazing quality, absurdres, [vibrant, very aesthetic, high contrast], masterful composition"}
add = "[light particles]" #@param {type:"string"}
rating = "safe sensitive" # @param ["safe","sensitive"] {"allow-input":true}
#@markdown ---
prompt = f"{style}, {char}, {rating}, {desc_char}, {pov}, {bg}, {add}"
negative_prompt = "femboy, text, watermark, bad anatomy, bad proportions, extra limbs, extra digit, extra legs, extra legs and arms, disfigured, missing arms, too many fingers, fused fingers, missing fingers, unclear eyes, watermark, username" #@param {type:'string'}
#@markdown ---
W = 960 #@param {type:"slider", min:512, max:2048, step:32}
H = 1312 #@param {type:"slider", min:512, max:2048, step:32}
guidance_scale = 5 #@param {type:'number'}
guidance_rescale = 0.9 #@param {type:'number'}
num_inference_steps = 32 #@param {type:"slider", min:10, max:40, step:1}
#@markdown ---

if len(char) > 10 or (char[:1] == "(" and char[-1:] == ")"):
    nama_char = char.split(", ")[1].split(" ")[0]
    jenis_char = char.split(", ")[0][1:]
outputname = "anime" #@param {type:'string'}
ratingX = rating.split(" ")[0]

outputbase = f"/content/outputs/{nama_char}_{ratingX}/{outputname}_{jenis_char}"
w = f"/content/outputs/{nama_char}_{ratingX}"
if not os.path.exists(w):
    os.makedirs(w)
rand = True #@param {type:'boolean'}
if rand:
    seed = 9999999 #@param {type:'integer'}
startNum = num

start_time = time.time()
array_time = []

for x in tqdm(range(amount), desc=f"Generating Images {num}"):
    gc.collect()
    torch.cuda.empty_cache()
    image_start_time = time.time()
    if rand:
        set_seed = random.randint(100000, 100000000)
    else:
        set_seed = seed
    output_file = f"{outputbase}_{num}_{set_seed}_{rating}.png"
    image = generate_image(pipe, prompt, negative_prompt, output_file, W, H, guidance_scale, guidance_rescale, num_inference_steps, set_seed)
    plt.imshow(image)
    image_end_time = time.time()
    array_time.append(image_end_time - image_start_time)
    num += 1

end_time = time.time()
total_time = end_time - start_time
log_metadata(f"prompt_{prompt_num}.txt", startNum, num - 1, outputbase, W, H, prompt, negative_prompt, guidance_scale, guidance_rescale, num_inference_steps, total_time)

gc.collect()
torch.cuda.empty_cache()

print(f"It took {math.floor(total_time/60)} minutes {math.floor(total_time%60)} seconds!")

In [ ]:
!git clone Stable_Diffusion/model_index/cagliostrolab_animagine-xl-4.0.json

In [ ]:
#@title +load_lora


lora_path = "/path/to/your/lora.safetensors"
pipe.load_lora_weights(lora_path)

In [ ]:

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

def img2img_generation(image, prompt, negative_prompt, num_inference_steps=25, guidance_scale=7.5, strength=0.7):
    images = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=image,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        strength=strength,
    ).images
    return images

# Contoh Penggunaan img2img
image_url = "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/sd_xl/input_1024x1024.png"
init_image = download_image(image_url)

prompt = "A futuristic city skyline, neon lights, cyberpunk style"
negative_prompt = "blurry, low quality, deformed"

generated_images = img2img_generation(
    image=init_image,
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=25,
    guidance_scale=7.5,
    strength=0.7
)

grid = image_grid(generated_images, rows=1, cols=1)
plt.imshow(grid)
plt.axis('off')
plt.show()

In [ ]:
#@title # +**remove_file(s)**
!rm "/content/"*.png
prompt_num=prompt_num+1

In [ ]:
!rm f"/content/{outputbase}{num-1}_{setseed}.png"

In [ ]:
#@title #**3.0 CONNECT & SAVE TO GOOGLE DRIVE**
from google.colab import drive
import datetime

drive.mount('/content/drive')

x = datetime.datetime.now().strftime("%Y_%m_%d")
!mkdir "/content/drive/MyDrive/Colab Notebooks/{x}"
!mkdir "/content/drive/MyDrive/Colab Notebooks/{x}/prompt{prompt_num}"

!mv "/content/outputs/" "/content/drive/MyDrive/Colab Notebooks/{x}/prompt{prompt_num}"
!mv "/content/"*.txt "/content/drive/MyDrive/Colab Notebooks/{x}/prompt{prompt_num}"

prompt_num = prompt_num + 1

In [ ]:
!mv /content/dir {dir}

In [ ]:
!mkdir outputs

In [ ]:
!mv "/content/"*.png "/content/outputs"

In [ ]:
!smi-nvidia

In [ ]:
#@title #**4.0 AUTOMATE PROMPTING (COMINGSOON)**

class PromptStableDiffusion:
    def __init__(self, subjek, gaya, latar_belakang, detail_tambahan):
        self.subjek = subjek
        self.gaya = gaya
        self.latar_belakang = latar_belakang
        self.detail_tambahan = detail_tambahan

    def buat_prompt(self):
        prompt = f"{self.subjek}, {self.gaya}, latar belakang {self.latar_belakang}, {self.detail_tambahan}"
        return prompt

prompt1 = PromptStableDiffusion(
    subjek="seorang wanita muda",
    gaya="lukisan cat minyak impresionis",
    latar_belakang="taman bunga matahari yang luas",
    detail_tambahan="matahari terbenam, burung-burung terbang di kejauhan"
)

prompt2 = PromptStableDiffusion(
    subjek="seekor naga raksasa",
    gaya="fantasi gelap",
    latar_belakang="kastil kuno yang hancur",
    detail_tambahan="langit mendung, petir menyambar"
)

daftar_prompt = [prompt1, prompt2]

for prompt_obj in daftar_prompt:
    print(prompt_obj.buat_prompt())



import json
daftar_prompt_json = [prompt_obj.__dict__ for prompt_obj in daftar_prompt]

with open("prompt_stable_diffusion.json", "w") as f:
    json.dump(daftar_prompt_json, f, indent=4)

print("\nPrompt telah disimpan dalam file prompt_stable_diffusion.json")

# **0.0 *~ ARCHIVE*** ~

In [ ]:
!pip install numba
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
#@markdown # Read Glosarium
!git clone https://github.com/FreNzQuiN/GenerativeAI/blob/main/Stable_Diffusion/cheatsheet.md

In [ ]:
!pip show diffusers

In [ ]:
#@title # **2.1** *ARCHIVED*
from compel import Compel, ReturnedEmbeddingsType
import matplotlib.pyplot as plt
from PIL import Image
import random, time, math, os
import torch, gc
from tqdm import tqdm


w = f"/content/outputs"
if not os.path.exists(w):
  !mkdir {w}

def generate_image(pipe, prompt, negative_prompt, output_path, width, height, guidance_scale, guidance_rescale, num_inference_steps, seed):
    """Menghasilkan gambar Stable Diffusion."""
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        seed=seed,
        width=width,
        height=height,
        guidance_scale=guidance_scale,
        guidance_rescale=guidance_rescale,
        target_size=(width, height),
        original_size=(4096, 4096),
        num_inference_steps=num_inference_steps,
        truncation=True
    ).images[0]
    image.save(output_path)
    return image

def log_metadata(file_path, start_num, end_num, output_base, width, height, prompt, negative_prompt, guidance_scale, guidance_rescale, num_inference_steps, elapsed_time):
    """Mencatat metadata ke dalam file teks."""
    with open(file_path, "a") as text_file:
        text_file.write(f"\n\nTITLE: {start_num}--{end_num} {output_base} {width}x{height}px\n + {prompt}\n - {negative_prompt}\n guide scale/rescale: {guidance_scale}/{guidance_rescale}\n steps: {num_inference_steps}\nIt took {math.floor(elapsed_time/60)} minutes {math.floor(elapsed_time%60)} seconds!")


amount = 50 #@param {type:'integer'}
#@markdown ---
char = "(1girl, goldenglow \\(arknights)\\, arknights \\(series)\\)" #@param {type:"string"}
desc_char = "close up, smirk, seducing expression, light blush, her outfit, mature teen, glow eyes, medium breast" #@param {type:'string'}
pov = "looking at viewer, from front, depth of field" # @param {"type":"string","placeholder":"dutch angle, looking at viewer, from front, depth of field, close up"}
bg = " colorful, indoors, room, natural scenery" # @param {"type":"string","placeholder":"colorful, indoors, natural scenery"}
style = "masterpiece, high score, best quality, amazing quality, absurdres" #@param {type:"string"}
add = "[light particles]" #@param {type:"string"}
rating = "safe sensitive" # @param ["safe","sensitive"] {"allow-input":true}
#@markdown ---
prompt = f"{style}, {char}, {rating}, {desc_char}, {pov}, {bg}, {add}"
negative_prompt = " text,watermark,bad anatomy,bad proportions,extra limbs,extra digit,extra legs,extra legs and arms,disfigured,missing arms,too many fingers,fused fingers,missing fingers,unclear eyes,watermark,username" #@param {type:'string'}
#@markdown ---
W = 960 #@param {type:"slider", min:512, max:2048, step:32}
H = 1312 #@param {type:"slider", min:512, max:2048, step:32}
guidance_scale = 5 #@param {type:'number'}
guidance_rescale = 0.9 #@param {type:'number'}
num_inference_steps = 32 #@param {type:"slider", min:10, max:40, step:1}
#@markdown ---

if len(char)>10 or (char[:1]=="(" and char[-1:]==")"):
  nama_char = char.split(", ")[1].split(" ")[0]
  jenis_char = char.split(", ")[0][1:]
outputname = "anime" #@param {type:'string'}
ratingX = rating.split(" ")[0]

outputbase = f"/content/outputs/{nama_char}_{ratingX}/{outputname}_{jenis_char}"
w = f"/content/outputs/{nama_char}_{ratingX}"
if not os.path.exists(w):
  !mkdir {w}
rand = True #@param {type:'boolean'}
if rand:
  seed = 9999999 #@param {type:'integer'}
startNum = num

start_time = time.time()
array_time = []

for x in tqdm(range(amount), desc=f"Generating Images {num}"):
    gc.collect()
    torch.cuda.empty_cache()
    image_start_time = time.time()
    if rand:
        set_seed = random.randint(100000, 100000000)
    else:
        set_seed = seed
    output_file = f"{outputbase}_{num}_{set_seed}_{rating}.png"
    image = generate_image(pipe, prompt, negative_prompt, output_file, W, H, guidance_scale, guidance_rescale, num_inference_steps, set_seed)
    plt.imshow(image)
    image_end_time = time.time()
    array_time.append(image_end_time - image_start_time)
    num += 1

end_time = time.time()
total_time = end_time - start_time
log_metadata(f"prompt{prompt_num}.txt", startNum, num - 1, outputbase, W, H, prompt, negative_prompt, guidance_scale, guidance_rescale, num_inference_steps, total_time)

gc.collect()
torch.cuda.empty_cache()

print(f"It took {math.floor(total_time/60)} minutes {math.floor(total_time%60)} seconds!")

In [ ]:
#@title #**2.1 PLAY TEXT2IMG SD** *ARCHIVED*
from compel import Compel, ReturnedEmbeddingsType
import matplotlib.pyplot as plt
from PIL import Image
import random
import time
import math
import torch, gc

textFile = open(f"prompt{prompt_num}.txt", "a")

amount = 5 #@param {type:'integer'}
prompt = "1girl, goldenglow \(arknights\), arknights \(series\), dutch angle, looking at viewer, cgdct, cute, solo, smile, light particles, casual, beautiful background, blush, outdoors, park, medium breast, from front, depth of field, very smooth line, year 2024, masterpiece, high score, great score, absurdres" #@param {type:'string'}
negative_prompt = "lowres, bad anatomy, bad hands, text, error, missing finger, extra digits, fewer digits, cropped, worst quality, low quality, low score, bad score, average score, signature, watermark, username, blurry" #@param {type:'string'}
outputbase = "/content/anime_girl" #@param {type:'string'}
startNum = num

# compel = Compel(
#   tokenizer=[pipe.tokenizer, pipe.tokenizer_2] ,
#   text_encoder=[pipe.text_encoder, pipe.text_encoder_2],
#   returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
#   requires_pooled=[False, True]
# )
# conditioning, pooled = compel(prompt)
W = 1024 #@param {type:'integer'}
H = 1424 #@param {type:'integer'}
guidance_scale = 5 #@param {type:'number'}
guidance_rescale = 0.9 #@param {type:'number'}
num_inference_steps = 28 #@param {type:'integer'}
rand = True #@param {type:'boolean'}
start = time.time()
arrayTime = []

for x in range(amount):
  gc.collect()
  torch.cuda.empty_cache()
  start = time.time()
  if rand == True:
    setseed = random.randint(100000,10000000)
  else:
    seed = 6 #@param {type:'integer'}
    setseed = seed
  image = pipe(
      # prompt_embeds=conditioning, pooled_prompt_embeds=pooled,
      prompt=prompt,
      negative_prompt=negative_prompt,
      seed=setseed,
      width=W,
      height=H,
      guidance_scale=guidance_scale,
      guidance_rescale=guidance_rescale,
      target_size=(W,H),
      original_size=(4096,4096),
      num_inference_steps=num_inference_steps,
      truncation=True
      ).images[0]
  output=f"{outputbase}{num}_{setseed}.png"
  image.save(output)
  image = Image.open(output)
  plt.imshow(image)
  end = time.time()
  length = end - start
  arrayTime.append(length)
  num=num+1

end = time.time()
length = end - start
print("It took", math.floor(length/60), "minutes", math.floor(length%60), "seconds!")
textFile.write(f"\n\nTITLE: {startNum}--{num-1} {outputbase} {W}x{H}px\n + {prompt}\n - {negative_prompt}\n guide scale/rescale: {guidance_scale}/{guidance_rescale}\n steps: {num_inference_steps}\nIt took {math.floor(length/60)} minutes {math.floor(length%60)} seconds!")
textFile.close()
gc.collect()
torch.cuda.empty_cache()